In [1]:
from torchvision import transforms, models
import matplotlib.pyplot as plt
from sklearn import metrics
from copy import deepcopy
from PIL import Image
import pandas as pd
import numpy as np
import torchvision
import random
import torch

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [3]:
tfs = transforms.Compose([
        transforms.ToTensor(),
        transforms.CenterCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [4]:
class Dataloader():
    
    def __init__(self, country, imagery_direc, scores_df, split, batch_size, tfs = None):
        """
        Arguments:
            country: one of ['mex', 'slv', 'peru', 'phl']
            imagery_direc: path to folder containing school imagery
            scores_df: path to CSV file with school IDs and test scroes
            split: train/test split, should be between .01 and 1, recommended is between .65 and .8
            batch_size: number of images in a batch
        """
        self.country = country
        self.imagery_direc = imagery_direc
        self.imagery = os.listdir(self.imagery_direc)
        self.imagery = [i for i in self.imagery if self.country in i]
        self.scores_df = pd.read_csv(scores_df)
        self.scores_df = self.scores_df[self.scores_df['country'] == self.country]
        self.split = split
        self.batch_size = batch_size
        
        if tfs is None:
            self.tfs = transforms.ToTensor()
        else:
            self.tfs = tfs
        
        # Load the data into a list with the format [(school_image, school_test_score), ...]
        self.data = self.load_data()
        
        # Split the data into training and validation sets
        self.train, self.val = self.test_train_split()
        
        
    def load_data(self):
        """
        Load the imagery into a list in the format: [(imager_tensor, test_score), ...]
        """
        data = []
        for col, row in self.scores_df.iterrows():
            school_id = str(row.school_id)
            test_score = row.y
            impath = [i for i in self.imagery if school_id in i]
            if len(impath) > 0:
                image = np.array(Image.open(self.imagery_direc + impath[0]))
                image = self.tfs(image)
                data.append((image, test_score))
        return data
                
        
    def test_train_split(self):
        """
        Split the data into 4 parts:
            x_train: imagery we use to train the model
            y_train: test_scores matched to the iamgery in x_train
            x_val: imagery used to test the model
            y_val: test_scores matched to the iamgery in x_val
        """
        train_num = int(len(self.data) * self.split)
        val_num = int(len(self.data) - train_num)

        all_indices = list(range(0, len(self.data)))
        train_indices = random.sample(range(len(self.data)), train_num)
        val_indices = list(np.setdiff1d(all_indices, train_indices))

        x_train, x_val = [self.data[i][0] for i in train_indices], [self.data[i][0] for i in val_indices]
        y_train, y_val = [self.data[i][1] for i in train_indices], [self.data[i][1] for i in val_indices]
        
        train = [(k,v) for k,v in zip(x_train, y_train)]
        val = [(k,v) for k,v in zip(x_val, y_val)]

        # Load the data into PyTorch's Dataloader format
        train = torch.utils.data.DataLoader(train, batch_size = self.batch_size, shuffle = True, drop_last = True)
        val = torch.utils.data.DataLoader(val, batch_size = self.batch_size, shuffle = True, drop_last = True)

        return train, val

In [5]:
COUNTRY = "bra"
BATCH_SIZE = 4
SPLIT = .75
IMAGERY_DIREC = "../../CCI/hmbaier/"
SCORES_DF = "./cci_final.csv"

In [6]:
data = Dataloader(country = COUNTRY, 
                  imagery_direc = IMAGERY_DIREC, 
                  scores_df = SCORES_DF,
                  split = SPLIT,
                  batch_size = BATCH_SIZE,
                  tfs = tfs)
all_data = data.data

In [7]:
# scores_df = pd.read_csv("./cci_final.csv")
# scores_df["y"] = np.where(scores_df["scaled_score"] > scores_df["scaled_score"].mean(), 1, 0)
# # scores_df = scores_df[scores_df["country"] == "bra"]
# scores_df

In [8]:
# scores_df.to_csv("./cci_final.csv", index = False)

In [9]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated
    
    
# train_features, train_labels = next(iter(all_data))
# out = torchvision.utils.make_grid(train_features)
# imshow(out, title = [i.item() for i in train_labels])

In [10]:
# We'll use the to keep track of our training stastics (i.e. running training loss, running validation loss, etc...)
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = round(self.sum / self.count, 4)

In [11]:
# Set up a basic off the shelf 
model = models.resnet18(pretrained = True)
model.fc = torch.nn.Linear(512, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load("./trained_phl_model.torch")["model_state_dict"]

model.load_state_dict(checkpoint)
model = model.to(device)

In [31]:
sm = torch.nn.Softmax()
trues, preds = [], []

for (inputs, targets) in all_data:
        
    inputs, targets = inputs.unsqueeze(0).to(device), torch.tensor(targets)
    outputs = model(inputs)
    
    _, pred = torch.max(sm(outputs), 1)
        
    trues.append(targets.item())
    preds.append(pred.item())

/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score


cm = confusion_matrix(trues, preds)
cm

array([[   0, 2140],
       [   0, 4601]])

In [36]:
accuracy_score(trues, preds)

0.6825396825396826